In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
Root_dir = '/content/drive/MyDrive/데이콘 대회/숫자/data/'
Train_dir = Root_dir + 'train.csv'

In [4]:
origin_data = pd.read_csv(Train_dir).set_index('id')

In [7]:
X=origin_data.iloc[:,2:].values.reshape(-1,28,28,1)
X = X/255.0
Y = origin_data['digit']

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.1, random_state=321)

In [17]:
def create_module(layer, channel_size, kernel_size):
    conv_layer = tf.keras.layers.Conv2D(channel_size, kernel_size= kernel_size, kernel_initializer = tf.keras.initializers.Orthogonal(), strides = 1, padding = 'same', activation = 'relu')(layer)
    batch_layer = tf.keras.layers.BatchNormalization()(conv_layer)
    conv_layer = tf.keras.layers.Conv2D(channel_size, kernel_size= kernel_size, kernel_initializer = tf.keras.initializers.Orthogonal(), strides = 1, padding = 'same', activation = 'relu')(batch_layer)
    batch_layer = tf.keras.layers.BatchNormalization()(conv_layer)
    pooling_layer = tf.keras.layers.MaxPooling2D((2, 2))(batch_layer)
    return pooling_layer

def create_model():
    inputs = tf.keras.layers.Input(shape = (X_train.shape[1:]))
    module_1 = create_module(inputs, 64, 2)
    module_2 = create_module(module_1, 64, 2)
    flatten_layer = tf.keras.layers.Flatten()(module_2)
    outputs=tf.keras.layers.Dense(10, activation='softmax')(flatten_layer)

    return tf.keras.models.Model(inputs=inputs, outputs=outputs)

In [18]:
model=create_model()
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
X_train=origin_data.iloc[:,2:].values.reshape(-1,28,28,1)
X_train = X_train/255.0
Y_train = origin_data['digit']

In [23]:
model_path=Root_dir + 'model_orthogonal/'

if not os.path.exists(model_path):
  os.mkdir(model_path)

checkpoint=tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path+'Val_{val_accuracy:.3f}.h5',
    monitor='val_accuracy', 
    save_best_only=True
)

early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    min_delta=0.1,
    patience=10,
    mode='max'
)

model.fit(
    X_train, y_train,
    epochs=100, batch_size=8,
    validation_split=0.2, shuffle=True, 
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/100
205/205 [==============================] - 1s 4ms/step - loss: 0.3460 - accuracy: 0.8974 - val_loss: 1.2874 - val_accuracy: 0.7268
Epoch 2/100
205/205 [==============================] - 1s 4ms/step - loss: 0.0894 - accuracy: 0.9707 - val_loss: 0.9801 - val_accuracy: 0.7634
Epoch 3/100
205/205 [==============================] - 1s 4ms/step - loss: 0.0244 - accuracy: 0.9915 - val_loss: 0.8541 - val_accuracy: 0.7902
Epoch 4/100
205/205 [==============================] - 1s 4ms/step - loss: 0.0087 - accuracy: 0.9982 - val_loss: 0.9002 - val_accuracy: 0.7805
Epoch 5/100
205/205 [==============================] - 1s 4ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.8968 - val_accuracy: 0.7805
Epoch 6/100
205/205 [==============================] - 1s 4ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.8949 - val_accuracy: 0.7805
Epoch 7/100
205/205 [==============================] - 1s 4ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.8954 - val_accuracy: 0.7878